
# Traditional GAN (MLP — **Unstable but Learning**, Fast) • Colab + Fast FID

Slightly stronger than the bare-bones MLP GAN yet **still prone to mode collapse**. Tuned for **speed** on A100/A10/L4/T4.
- MLP **Generator/Discriminator** (no convs, no BatchNorm)
- **Non-saturating** G loss, **Adam** with unbalanced LRs (G > D)
- **Mild label smoothing** + **tiny instance noise** (annealed)
- **Mixed precision** training + **XLA** JIT (FID runs in float32)
- 64×64 pipeline, persistent **4×4** sample grids (~330 px), **fast FID**, checkpoints, CSV, **loss & FID plots**, **GIF**


In [ ]:

# =====================
# Configuration (fast defaults)
# =====================
import os

# Data & output
DATA_DIR            = "monet_jpg"  # Change if needed
OUTPUT_DIR          = "outputs_gan_mlp_unstable_learning_fast"
SAMPLES_DIR         = os.path.join(OUTPUT_DIR, "samples")
CKPT_DIR            = os.path.join(OUTPUT_DIR, "checkpoints")
METRICS_CSV         = os.path.join(OUTPUT_DIR, "metrics_history.csv")

# Image & training
IMG_SIZE            = 64
CHANNELS            = 3
BATCH_SIZE          = 64
BUFFER_SIZE         = 2000
EPOCHS              = 80

# Per-epoch work (reduced for speed)
REPEAT_DATASET      = True
STEPS_PER_EPOCH     = 120         # was 300

# Optim (unbalanced to keep instability)
GEN_LR              = 2e-3        # stronger G
DISC_LR             = 3e-4        # weaker D

# Labels + instance noise (annealed)
REAL_LABEL_MIN      = 0.85
REAL_LABEL_MAX      = 1.00
FAKE_LABEL_MIN      = 0.00
FAKE_LABEL_MAX      = 0.15
INSTANCE_NOISE_START= 0.05
INSTANCE_NOISE_END  = 0.00

# Display / saving
SAMPLES             = 16          # 4x4 grid
GRID_NROW, GRID_NCOL= 4, 4
DISPLAY_WIDTH_PX    = 330
SAVE_GRID_EVERY     = 1
DISPLAY_GRID_EVERY  = 1

# Caching
CACHE_DATASET       = True
CACHE_PATH          = ""

# FID (fast & memory-safe)
COMPUTE_FID_EVERY   = 10          # was 5
REAL_FID_SAMPLES    = 500
FAKE_FID_SAMPLES    = 100         # was 200
FID_GEN_BATCH       = 128         # was 64
FID_ACT_BATCH       = 1024        # was 512

# Precision (allow mixed precision on supported GPUs)
FORCE_FLOAT32_TRAIN = False

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(SAMPLES_DIR, exist_ok=True)
os.makedirs(CKPT_DIR, exist_ok=True)


In [ ]:

# =====================
# Imports & Environment
# =====================
import glob, time, math
import numpy as np
import imageio
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print("TensorFlow:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))


In [ ]:

# =====================
# Mixed Precision (A100/A10/L4/T4) + XLA for speed
# =====================
from tensorflow.keras import mixed_precision
use_mixed_precision = False

gpus = tf.config.list_physical_devices('GPU')
gpu_name = ""
if gpus:
    try:
        details = tf.config.experimental.get_device_details(gpus[0])
        gpu_name = details.get('device_name', '')
    except Exception:
        pass

if not FORCE_FLOAT32_TRAIN and gpus and any(x in (gpu_name or '').upper() for x in ["A100", "A10", "L4", "T4"]):
    mixed_precision.set_global_policy('mixed_float16')
    use_mixed_precision = True
else:
    mixed_precision.set_global_policy('float32')

try:
    tf.config.optimizer.set_jit(True)   # XLA on
    xla_state = "ON"
except Exception:
    xla_state = "OFF"

print("Policy:", mixed_precision.global_policy())
print("XLA JIT:", xla_state, "| GPU:", gpu_name or "None")


In [ ]:

# =====================
# Single-Device Strategy
# =====================
strategy = tf.distribute.get_strategy()
print("Using default single-device strategy.")



### (Optional) Copy dataset from Drive to local disk (faster I/O in Colab)
Uncomment and run this once if your images are in Drive.


In [ ]:

# !rsync -ah --info=progress2 "/content/drive/MyDrive/monet_jpg/" "/content/monet_jpg/"
# DATA_DIR = "/content/monet_jpg"


In [ ]:

# =====================
# Data Loader (faster JPEG path)
# =====================
AUTOTUNE = tf.data.AUTOTUNE

def load_and_preprocess(path):
    img_bytes = tf.io.read_file(path)
    # Fast decode for JPEG; generic fallback otherwise
    is_jpeg = tf.strings.regex_full_match(tf.strings.lower(path), ".*\.(jpg|jpeg)$")
    def _decode_jpeg(): return tf.io.decode_jpeg(img_bytes, channels=3, dct_method='INTEGER_ACCURATE')
    def _decode_any():  return tf.image.decode_image(img_bytes, channels=3, expand_animations=False)
    img = tf.cond(is_jpeg, _decode_jpeg, _decode_any)
    img.set_shape([None, None, 3])
    img = tf.image.convert_image_dtype(img, tf.float32)  # [0,1]
    h = tf.shape(img)[0]; w = tf.shape(img)[1]
    side = tf.minimum(h, w)
    img = tf.image.crop_to_bounding_box(img, (h-side)//2, (w-side)//2, side, side)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE), method="bicubic")
    img = (img - 0.5) * 2.0  # [-1,1]
    return img

def make_dataset(data_dir, batch_size):
    patterns = ["*.jpg", "*.jpeg", "*.png"]
    files = []
    for p in patterns:
        files.extend(glob.glob(os.path.join(data_dir, p)))
    if not files:
        raise FileNotFoundError(f"No images found under '{data_dir}'. Place JPG/PNG files there.")
    paths = tf.constant(files)
    ds = tf.data.Dataset.from_tensor_slices(paths)
    ds = ds.map(load_and_preprocess, num_parallel_calls=AUTOTUNE)
    if CACHE_DATASET:
        ds = ds.cache(CACHE_PATH) if CACHE_PATH else ds.cache()
    ds = ds.shuffle(min(len(files), BUFFER_SIZE), reshuffle_each_iteration=True)
    ds = ds.batch(batch_size, drop_remainder=True)
    if REPEAT_DATASET:
        ds = ds.repeat()
    ds = ds.prefetch(AUTOTUNE)
    return ds, len(files)

train_dataset, num_files = make_dataset(DATA_DIR, BATCH_SIZE)
print(f"Found {num_files} images; batch size {BATCH_SIZE}; IMG_SIZE {IMG_SIZE}.")


In [ ]:

# =====================
# FID Utilities (GPU Inception, float32) — memory safe & fast
# =====================
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras import mixed_precision

try:
    from scipy.linalg import sqrtm as _scipy_sqrtm
    _HAS_SCIPY = True
except Exception:
    _HAS_SCIPY = False

_prev = mixed_precision.global_policy()
mixed_precision.set_global_policy('float32')
_dev = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'
with tf.device(_dev):
    _inception = InceptionV3(include_top=False, weights='imagenet', pooling='avg', input_shape=(299,299,3))
_inception.trainable = False
mixed_precision.set_global_policy(_prev)

REAL_STATS_CACHE = None

def _to_uint8(x01):
    x = tf.clip_by_value(x01, 0.0, 1.0)
    return tf.cast(x * 255.0, tf.uint8)

def _prep(uint8_batch):
    x = tf.cast(uint8_batch, tf.float32)
    x = tf.image.resize(x, (299,299), method='bicubic')
    return preprocess_input(x)

@tf.function(jit_compile=False)
def _inception_acts(x_f32):
    return _inception(x_f32, training=False)

def _activations_batched(uint8_images, batch_size=FID_ACT_BATCH):
    n = int(uint8_images.shape[0])
    outs = []
    dev = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'
    for i in range(0, n, batch_size):
        x = _prep(uint8_images[i:i+batch_size])
        with tf.device(dev):
            acts = _inception_acts(x)
        outs.append(acts)
    return tf.concat(outs, axis=0)

def _mu_sigma(acts: tf.Tensor):
    a = acts.numpy()
    return a.mean(0), np.cov(a, rowvar=False)

def _fid(mu1, s1, mu2, s2):
    if _HAS_SCIPY:
        covmean = _scipy_sqrtm(s1.dot(s2)).real
    else:
        covmean = tf.linalg.sqrtm(tf.convert_to_tensor(s1 @ s2, tf.float64))
        covmean = tf.math.real(covmean).numpy()
    diff = mu1 - mu2
    return float(diff.dot(diff) + np.trace(s1 + s2 - 2.0 * covmean))

def sample_real_uint8(ds, n):
    out, cnt = [], 0
    for b in ds:
        x = (b + 1.0) * 0.5
        out.append(_to_uint8(x)); cnt += x.shape[0]
        if cnt >= n: break
    return tf.concat(out, 0)[:n]


In [ ]:

# =====================
# Sampling Utilities (4x4 small display)
# =====================
from IPython.display import display, Image as IPyImage

def make_grid(imgs, nrow, ncol):
    rows = []
    for r in range(nrow):
        row = np.concatenate([imgs[r*ncol + c] for c in range(ncol)], axis=1)
        rows.append(row)
    return np.concatenate(rows, axis=0)

def save_and_show_grid(imgs_01, epoch, secs=None, nrow=GRID_NROW, ncol=GRID_NCOL, out_dir=SAMPLES_DIR, prefix=""):
    grid = make_grid(imgs_01, nrow, ncol)
    os.makedirs(out_dir, exist_ok=True)
    raw_path = os.path.join(out_dir, f"{prefix}epoch_{epoch:04d}.png")
    imageio.imwrite(raw_path, (grid * 255).astype(np.uint8))

    fig = plt.figure(figsize=(2.2, 2.2), dpi=150)   # ~330 px
    plt.imshow(grid)
    title = f"Epoch {epoch}" + (f" | {secs:.1f}s" if secs is not None else "")
    plt.title(title, fontsize=8)
    plt.axis('off')
    ann_path = os.path.join(out_dir, f"{prefix}epoch_{epoch:04d}_annotated.png")
    plt.savefig(ann_path, bbox_inches='tight', pad_inches=0.02)
    plt.close(fig)
    display(IPyImage(filename=ann_path, width=DISPLAY_WIDTH_PX))
    return raw_path, ann_path


In [ ]:

# =====================
# MLP GAN (slightly larger, still small/unstable)
# =====================
LATENT_DIM = 64
FLAT_DIM   = IMG_SIZE * IMG_SIZE * CHANNELS  # 12288

def make_mlp_generator():
    z_in = layers.Input(shape=(LATENT_DIM,))
    x = layers.Dense(512, activation='relu')(z_in)
    x = layers.Dense(384, activation='relu')(x)           # trimmed for speed
    x = layers.Dense(FLAT_DIM, activation='tanh')(x)      # [-1,1]
    x = layers.Reshape((IMG_SIZE, IMG_SIZE, CHANNELS))(x)
    return keras.Model(z_in, x, name="G_mlp_unstable_fast")

def make_mlp_discriminator():
    x_in = layers.Input(shape=(IMG_SIZE, IMG_SIZE, CHANNELS))
    x = layers.Flatten()(x_in)
    x = layers.Dense(192)(x); x = layers.LeakyReLU(0.2)(x)
    x = layers.Dense(96)(x);  x = layers.LeakyReLU(0.2)(x)
    x = layers.Dense(1, activation=None)(x)  # logits
    return keras.Model(x_in, x, name="D_mlp_unstable_fast")

tf.keras.backend.clear_session()
with strategy.scope():
    G = make_mlp_generator()
    D = make_mlp_discriminator()
    gen_opt  = keras.optimizers.Adam(GEN_LR,  beta_1=0.5, beta_2=0.9)
    disc_opt = keras.optimizers.Adam(DISC_LR, beta_1=0.5, beta_2=0.9)
bce_logits = keras.losses.BinaryCrossentropy(from_logits=True)
print(G.summary()); print(D.summary())


In [ ]:

# =====================
# Training step (non-saturating) + label smoothing + instance noise
# =====================
LATENTS_FIXED = tf.random.normal([SAMPLES,  LATENT_DIM])

@tf.function(jit_compile=True)
def train_step(real_images, sigma):
    b = tf.shape(real_images)[0]

    # --- Update D ---
    z = tf.random.normal([b, LATENT_DIM])
    with tf.GradientTape() as tape:
        fake = G(z, training=True)

        # instance noise
        if sigma > 0:
            noise_r = tf.random.normal(tf.shape(real_images), stddev=sigma, dtype=real_images.dtype)
            noise_f = tf.random.normal(tf.shape(fake),        stddev=sigma, dtype=fake.dtype)
            real_in = tf.clip_by_value(real_images + noise_r, -1.0, 1.0)
            fake_in = tf.clip_by_value(fake + noise_f,        -1.0, 1.0)
        else:
            real_in, fake_in = real_images, fake

        real_logits = D(real_in, training=True)   # (b,1)
        fake_logits = D(fake_in, training=True)   # (b,1)

        # label smoothing with correct shapes
        real_t = tf.random.uniform(tf.shape(real_logits),
                                   minval=REAL_LABEL_MIN, maxval=REAL_LABEL_MAX,
                                   dtype=real_logits.dtype)
        fake_t = tf.random.uniform(tf.shape(fake_logits),
                                   minval=FAKE_LABEL_MIN,  maxval=FAKE_LABEL_MAX,
                                   dtype=fake_logits.dtype)

        d_loss_real = bce_logits(real_t, real_logits)
        d_loss_fake = bce_logits(fake_t, fake_logits)
        d_loss = d_loss_real + d_loss_fake
    grads = tape.gradient(d_loss, D.trainable_variables)
    disc_opt.apply_gradients(zip(grads, D.trainable_variables))

    # --- Update G (non-saturating) ---
    z2 = tf.random.normal([b, LATENT_DIM])
    with tf.GradientTape() as tape:
        fake2 = G(z2, training=True)
        fake_logits2 = D(fake2, training=True)
        g_loss = bce_logits(tf.ones_like(fake_logits2), fake_logits2)  # target 1
    grads = tape.gradient(g_loss, G.trainable_variables)
    gen_opt.apply_gradients(zip(grads, G.trainable_variables))

    return g_loss, d_loss

def generate_preview_samples():
    imgs = G(LATENTS_FIXED, training=False)
    imgs = (imgs + 1.0) * 0.5
    return tf.clip_by_value(imgs, 0.0, 1.0).numpy()


In [ ]:

# =====================
# FID wrappers
# =====================
def sample_fake_uint8_from_gan(n, gen_batch=FID_GEN_BATCH):
    outs = []
    left = n
    while left > 0:
        m = min(gen_batch, left)
        z = tf.random.normal([m, LATENT_DIM])
        imgs = G(z, training=False)
        imgs = (imgs + 1.0) * 0.5
        outs.append(tf.cast(tf.clip_by_value(imgs, 0.0, 1.0) * 255.0, tf.uint8))
        left -= m
    return tf.concat(outs, 0)

def compute_fid_fast_gan(ds, num_real=REAL_FID_SAMPLES, num_fake=FAKE_FID_SAMPLES,
                         act_batch=FID_ACT_BATCH, gen_batch=FID_GEN_BATCH):
    global REAL_STATS_CACHE
    if REAL_STATS_CACHE is None:
        x = sample_real_uint8(ds, num_real)
        a = _activations_batched(x, batch_size=act_batch)
        REAL_STATS_CACHE = _mu_sigma(a)
    mu_r, s_r = REAL_STATS_CACHE
    x_fake = sample_fake_uint8_from_gan(num_fake, gen_batch=gen_batch)
    a_fake = _activations_batched(x_fake, batch_size=act_batch)
    mu_f, s_f = _mu_sigma(a_fake)
    return _fid(mu_r, s_r, mu_f, s_f)


In [ ]:

# =====================
# Checkpoints
# =====================
ckpt = tf.train.Checkpoint(G=G, D=D, gen_opt=gen_opt, disc_opt=disc_opt)
manager = tf.train.CheckpointManager(ckpt, CKPT_DIR, max_to_keep=3)
if manager.latest_checkpoint:
    ckpt.restore(manager.latest_checkpoint)
    print(f"Restored from {manager.latest_checkpoint}")
else:
    print("Initializing from scratch.")


In [ ]:

# =====================
# Training Loop + CSV + grids + FID
# =====================
import pandas as pd

def sigma_for_epoch(ep):
    if EPOCHS <= 1: return float(INSTANCE_NOISE_END)
    t = (ep - 1) / (EPOCHS - 1)
    return float((1.0 - t) * INSTANCE_NOISE_START + t * INSTANCE_NOISE_END)

history = []
print("Starting MLP GAN (unstable-but-learning, fast)...")
for epoch in range(1, EPOCHS+1):
    t0 = time.time()
    step = 0
    last_g, last_d = None, None
    sigma = sigma_for_epoch(epoch)
    for real_batch in train_dataset:
        g_loss, d_loss = train_step(real_batch, tf.constant(sigma, dtype=real_batch.dtype))
        last_g, last_d = g_loss, d_loss
        step += 1
        if step >= STEPS_PER_EPOCH: break
    secs = time.time() - t0

    # grid
    if (epoch % SAVE_GRID_EVERY) == 0:
        imgs_01 = generate_preview_samples()
        save_and_show_grid(imgs_01, epoch, secs)

    fid_val = None
    if COMPUTE_FID_EVERY and (epoch % COMPUTE_FID_EVERY) == 0:
        try:
            t1 = time.time()
            fid_val = compute_fid_fast_gan(train_dataset)
            print(f"FID @ epoch {epoch}: {fid_val:.2f} (took {time.time()-t1:.1f}s)")
        except Exception as e:
            print(f"FID computation failed: {e}")

    if (epoch % 10) == 0:
        path = manager.save(); print(f"Saved checkpoint at {path}")

    row = {"epoch": epoch,
           "g_loss": float(last_g.numpy()) if last_g is not None else np.nan,
           "d_loss": float(last_d.numpy()) if last_d is not None else np.nan,
           "secs": secs, "fid": fid_val, "sigma": sigma}
    history.append(row)
    try:
        pd.DataFrame(history).to_csv(METRICS_CSV, index=False)
    except Exception as e:
        print(f"(Could not save metrics CSV: {e})")

print("Training complete.")


In [ ]:

# =====================
# Plots: Loss & FID
# =====================
import pandas as pd
import matplotlib.pyplot as plt
import os

if os.path.exists(METRICS_CSV):
    df = pd.read_csv(METRICS_CSV)
    if not df.empty:
        # Loss
        plt.figure()
        plt.plot(df["epoch"], df["g_loss"], label="G loss")
        plt.plot(df["epoch"], df["d_loss"], label="D loss")
        plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title("GAN Loss vs Epoch"); plt.legend(); plt.show()

        # FID
        if "fid" in df.columns and df["fid"].notna().any():
            df_fid = df.dropna(subset=["fid"])
            plt.figure()
            plt.plot(df_fid["epoch"], df_fid["fid"])
            plt.xlabel("Epoch"); plt.ylabel("FID"); plt.title("FID vs Epoch"); plt.show()
    else:
        print("Metrics CSV is empty.")
else:
    print("Metrics CSV not found.")


In [ ]:

# =====================
# Make & show a GIF from saved sample grids
# =====================
import os, glob, imageio
from IPython.display import display, Image as IPyImage

annot = sorted(glob.glob(os.path.join(SAMPLES_DIR, "epoch_*_annotated.png")))
raw   = sorted(glob.glob(os.path.join(SAMPLES_DIR, "epoch_*.png")))
files = annot if annot else raw
assert files, f"No sample images found under {SAMPLES_DIR}. Run training first."

GIF_PATH   = os.path.join(OUTPUT_DIR, "training_recap.gif")
FRAME_SKIP = 1
DURATION   = 0.15

with imageio.get_writer(GIF_PATH, mode="I", duration=DURATION) as writer:
    for i, f in enumerate(files):
        if (i % FRAME_SKIP) != 0:
            continue
        writer.append_data(imageio.imread(f))

display(IPyImage(filename=GIF_PATH, width=DISPLAY_WIDTH_PX))
print(f"Saved GIF: {GIF_PATH}  |  frames: {len(files[::FRAME_SKIP])}  |  duration/frame: {DURATION}s")
